In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの分析にあたって、以下の観点から重要なインサイトを得ることができます。

### 1. カテゴリー別の売上分析
- **収益性の比較**: 食品、衣服、日用品の各カテゴリごとに総売上高、総コスト、利益を計算し、どのカテゴリが最も利益を生んでいるかを確認します。
- **売上トレンド**: 各カテゴリの売上が月ごとにどう変化しているかトレンドを分析し、季節性や特定のキャンペーンが影響しているかを見極めます。

### 2. 商品別売上分析
- **人気商品**: 最も売上を上げた商品や、数量が最も多く売れた商品を特定し、顧客の購買傾向を理解します。
- **単価と数量の相関**: 高単価の商品が多く売上を上げているのか、低単価で大量に購入される商品が多いのかを分析します。

### 3. 利益分析
- **利益率の計算**: 各商品の利益（売上 - 原価）を計算し、利益率を算出します。利益率が高い商品を特定することで、重点的にマーケティング施策を行う対象を見つけられます。
  
### 4. 売上日分析
- **日別売上の変動**: 売上が増加した特定の日や週、月のトレンドを分析し、ピークタイムや閑散期を把握します。

### 5. 顧客セグメンテーション
- 購買頻度や客単価に基づいて顧客をセグメント化し、ターゲットマーケティングのアプローチを考えます。

### 例: 分析結果のサンプル
- **食品カテゴリ**: 売上が高いが、利益率が低い場合、キャンペーンやプロモーションを検討。
- **衣服カテゴリ**: 利益率が高いと分析された場合、類似商品を拡充する戦略を考える。

### 次のステップ
- 各観点から得られたインサイトをもとに、具体的な施策を若干の仮説検証を通じて立案。
- 例えば、特定の曜日や月で売上が伸びている場合、その期間に特化した広告キャンペーンを実施するなど、データ駆動型の意思決定を行います。

このように、売上データの詳細な分析を通じて、企業の成長をサポートするための具体的な施策や戦略を導出することが可能です。データの詳細や外部要因をも考慮に入れながら、実行可能なアクションを提示することが重要です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0            売上データの分析にあたって、以下の観点から重要なインサイトを得ることができます。
1                                                    
2                                  ### 1. カテゴリー別の売上分析
3   - **収益性の比較**: 食品、衣服、日用品の各カテゴリごとに総売上高、総コスト、利益を計...
4   - **売上トレンド**: 各カテゴリの売上が月ごとにどう変化しているかトレンドを分析し、季...
5                                                    
6                                      ### 2. 商品別売上分析
7   - **人気商品**: 最も売上を上げた商品や、数量が最も多く売れた商品を特定し、顧客の購買...
8   - **単価と数量の相関**: 高単価の商品が多く売上を上げているのか、低単価で大量に購入さ...
9                                                    
10                                        ### 3. 利益分析
11  - **利益率の計算**: 各商品の利益（売上 - 原価）を計算し、利益率を算出します。利益...
12                                                   
13                                       ### 4. 売上日分析
14  - **日別売上の変動**: 売上が増加した特定の日や週、月のトレンドを分析し、ピークタイム...
15                                                   
16                                 ### 5. 顧客セグメンテーション
17  - 購買頻度や客単価に基づいて顧客をセグメント化

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [ ]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")